In [12]:
import pandas as pd
import re
import json
import numpy as np

In [2]:
with open('aggregate_results.json') as f:
  aggregate_results = json.load(f)

In [3]:
# General functions used to develop extractors

def general_splitter(x, patterns):
    new_string = str(x)
    first_pass = new_string.split(patterns[0], 1)[1]
    second_pass = first_pass.split(patterns[1], 1)[0]
    return(second_pass)

def pattern_sequencer(x, patterns, potential_patterns):
    for p in range(0, len(potential_patterns)):
            if str(x).find(potential_patterns[p][0]) != -1:
                patterns = potential_patterns[p]
                break
            else:
                patterns = ["No Matching Pattern", ""]
    return(patterns)
    
def general_extractor(x, patterns, potential_patterns, keyword):
    if str(x).find(keyword) != -1:
        new_patterns = pattern_sequencer(x, patterns, potential_patterns)
        if new_patterns[0] != "No Matching Pattern":
            return(general_splitter(x, new_patterns))
        else:
            return("Bad Pattern")
    else:
        return("Can't find keyword")

In [189]:
firm_patterns = ["Matter of:\xa0</strong> ", "</p>"]
potential_firm_patterns = [["Matter of:\xa0</strong> ", "</p>\n<p><strong>File:"],
                          ["Matter of:</strong> ", "</p>"],
                          ["Matter of:\xa0 </strong>", "</p>"],
                          ["Matter of:</strong>", "</p>"],
                          ["Matter of: \xa0</strong> ","</p>"],
                          ["Matter of:\xa0\xa0\xa0\xa0\xa0 </strong>", "</p>"],
                          ["Matter of: </strong>", "</p>"],
                          ["<p class=\"MatterOf\">\n\t\tMatter of: ", "</p>"],
                          ["Matter of:      </strong>", "</p>"]]

date_patterns = ["\n<p><strong>Date:</strong>\xa0 ", "</p>"]
potential_date_patterns = [["<p><strong>Date:</strong>\xa0 ", "</p>"],
                           ["<p><strong>Date:\xa0</strong> ", "</p>"],
                          ["<p>\n<strong>Date:</strong>\xa0 ", "</p>"],
                          ["<p><strong>Date:</strong> ", "</p>"],
                          ["Date:</strong>", "</p>"],
                          ["Date:\xa0 </strong>", "</p>"],
                          ["Date:\xa0</strong> ", "</p>"],
                          ["Date:\xa0 \xa0</strong> ", "</p>"],
                          ["Date:</strong>\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0", "</p>"],
                          ["Date:</strong> ", "</p>"],
                          ["Date:\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 </strong>", "</p>"],
                          ["Date: </strong>", "</p>"],
                          ["<p class=\"Date\">\n\t\tDate: ", "</p>"]]

paragraph_patterns = ["</p>\n<div>\n<p>DECISION</p>\n</div>\n<p>" , "</p>\n"]
potential_paragraph_patterns = [["</p>\n<div>\n<p>DECISION</p>\n</div>\n<p>" , "</p>\n"],
                                ["<div>\n<p><strong>DECISION</strong></p>\n</div>\n<p>", "</p>"],
                               ["DECISION</p>\n<p>", "</p>"],
                               ["DECISION</p>\n</div>\n<p>\n\t\t", "</p>"],
                                ["DECISION</strong></p>\n<p>\n\t\t", "</p>\n<p>\n\t\t"],
                               ["DECISION</p>\n</div>\n<p>\n\t\t", "\xa0</p>\n<p>\n\t\t"],
                               ["DECISION</strong></div>\n<p>", "</p>"],
                               ["DECISION</strong></p>\n<p>", "</p>"],
                               ["DECISION</p>\n</div>\n<p>", "</p>"],
                               ["DECISION\xa0 </strong></p>\n</div>\n<p>\n\t\t", "</p>"],
                               ["DECISION</strong></p>\n</div>\n<p>\n\t\t", "</p>"],
                               ["DECISION\xa0</p>\n</div>\n<p>\n\t\t", "</p>"],
                               ["DECISION</strong></p>\n</div>\n<p>\n\t", "</p>"],
                               ["DECISION</p>\n</div>\n<p class=\"MsoBodyText\">\n\t\t", "</p>"],
                               ["DECISION</p>\n</div>\n<p class=\"MsoNormal\">\n\t\t", "</p>"],
                               ["DECISION</strong></p>\n</div>\n<p class=\"MsoNormal\">\n\t\t", "</p>"],
                               ["DECISION</p>\n</div>\n<p class=\"Date\">\n\t\t", "</p>"],
                               ["DECISION</strong></p>\n<p class=\"MsoNormal\">\n\t\t\t", "</p>"],
                               ["DECISION</p>\n</div>\n<p class=\"MsoBodyText\">\n\t", "</p>"],
                               ["DECISION</p>\n</div>\n<p class=\"MatterOf\">\n\t\t", "</p>"],
                               ["DECISION</p>\n</div>\n<p class=\"Default\">\n\t\t", "</p>"],
                               ["DECISION</p>\n</div>\n<p class=\"Decision1\">\n\t\t", "</p>"],
                               ["DECISION</p>\n<p class=\"Decision1\">\n\t\t\t", "</p>"],
                               ["DECISION</strong></p>\n<p class=\"MsoBodyText\">\n\t\t", "</p>"],
                               ["DECISION</p>\n</div>\n<p class=\"matterof0\">\n\t\t", "</p>"],
                               ["DECISION</p>\n</div>\n<p class=\"MsoNormal\">", "</p>"],
                               ["DECISION</p></div><p class=\"MsoBodyText\"> </p><p class=\"MsoBodyText\">", "</p>"],
                               ["DECISION</p></div><p class=\"MsoBodyText\"> </p><p class=\"MsoNormal\">", "</p>"],
                                ["DECISION</p></div><p class=\"MsoBodyTextCxSpFirst\"> </p><p class=\"MsoBodyTextCxSpMiddle\">", "</p>"]]

status_patterns = ['<div class="status highlighted-status">\n', '.\n      </div>']
potential_status_patterns = [['<div class="status highlighted-status">\n', '.\n      </div>']]

highlights_patterns = ["<html><body><p>\n\t", "</p>"]
potential_highlights_patterns = [["<html><body><p>", "</p>"], 
                                 ["<html><body><p>\n\t", "</p>"],
                                ["<html><body><p align=\"LEFT\">", "</p>"],
                                ["<html><body><div>\n\t", "</p>"]]

In [190]:
good_df = pd.DataFrame()

for d in range(0, 2500):
    document = aggregate_results[str(d)]['document']
    stat = aggregate_results[str(d)]['status']
    high = aggregate_results[str(d)]['highlights']
    firm = general_extractor(document, firm_patterns, potential_firm_patterns, keyword = "Matter of:")
    date = general_extractor(document, date_patterns, potential_date_patterns, keyword = "Date:")
    status = general_extractor(stat, status_patterns, potential_status_patterns, keyword = "status")
    decision_paragraph = general_extractor(document, paragraph_patterns, potential_paragraph_patterns, keyword = "DECISION")
    highlights = general_extractor(high, highlights_patterns, potential_highlights_patterns, keyword = "Highlights")
    series = {"Firm" :str(firm), "Date" : str(date), "Status" : str(status), "Decision_Paragraph" : str(decision_paragraph), "Highlights_Paragraph" : str(highlights), "URL" : aggregate_results[str(d)]['url']}
    good_df = good_df.append(series, ignore_index = True)

In [191]:
date_bad_pattern = good_df.Date == "Bad Pattern"
firm_bad_pattern = good_df.Firm == "Bad Pattern"
status_bad_pattern = good_df.Status == "Bad Pattern"
decision_bad_pattern = good_df.Decision_Paragraph == "Bad Pattern"
highlights_bad_pattern = good_df.Highlights_Paragraph == "Bad Pattern"

check_df = good_df[decision_bad_pattern | firm_bad_pattern | date_bad_pattern | decision_bad_pattern | highlights_bad_pattern]
check_df

,Date,Decision_Paragraph,Firm,Highlights_Paragraph,Status,URL
2439,"December 30, 2013",Bad Pattern,"Superlative Technologies, Inc.","Superlative Technologies, Inc. (SuprTEK), of A...",We deny the protest,https://www.gao.gov/products/b-408941
2449,"December 31, 2013",Bad Pattern,IJC Corporation,"IJC Corporation (IJC), a service-disabled vete...",We deny the protest,https://www.gao.gov/products/b-408950
2455,"December 16, 2013",Bad Pattern,"CACI-WGI, Inc.","CACI-WGI, Inc., of Vienna, Virginia, protests ...",We deny the protest,https://www.gao.gov/products/b-408520.2
2464,"December 6, 2013",Bad Pattern,"Mission Essential Personnel, LLC","Mission Essential Personnel, LLC (MEP), of Col...",We deny the protest,https://www.gao.gov/products/b-408788
2465,"November 13, 2013",Bad Pattern,BAE Systems Information and Electronic Systems...,BAE Systems Information and Electronic Systems...,We sustain the protest,https://www.gao.gov/products/b-408565%2cb-4085...
2469,"December 12, 2013",Bad Pattern,"Portfolio Management Solutions, LLC;<strong> <...","Portfolio Management Solutions, LLC; of Washin...",We deny the protest,https://www.gao.gov/products/b-408846%2cb-4088...
2499,"November 1, 2013",Bad Pattern,"CACI Technologies, Inc.","CACI Technologies, Inc., of Chantilly, Virgini...",We deny the protest,https://www.gao.gov/products/b-408552


In [192]:
indices = check_df.index.values

test_doc = aggregate_results[str(indices[0])]['document']

test_doc

'[<div class="js-endpoint-view-decision field field--name-field-html-block field--type-text-long field--label-above">\n<header class="field__label">View Decision</header>\n<div class="field-items-wrapper">\n<div class="field__item"><div class="WordSection1"><p class="DecisionT">DOCUMENT FOR PUBLIC RELEASE<br/>The decision issued on the date below was subject to a GAO Protective Order. This redacted version has been approved for public release.</p><p class="DecisionT"><strong>Decision</strong></p><p class="MsoBodyText"> </p><p class="MsoBodyText"> </p><p class="MatterOf"><strong>Matter of: </strong>Superlative Technologies, Inc.</p><p class="MatterOf"> </p><p class="File"><strong>File: </strong>B-408941</p><p class="MsoBodyText"> </p><p class="Date"><strong>Date:</strong> December 30, 2013</p><p class="MsoBodyText"> </p><div><p class="Parties">Kenneth D. Brody, Esq., and Thomas K. David, Esq., David, Brody &amp; Dondershine, LLP, for the protester.<br/>Lee Dougherty, Esq., and Katherine

array([1829, 1847, 1875, 1896, 1949, 1954, 1970, 1988, 1993], dtype=int64)